In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import pandasql as ps
from pandasql import sqldf

sns.set(font_scale=3)
sns.set(color_codes=True)
%matplotlib inline
import matplotlib.pyplot as plt


plt.rcParams["figure.figsize"] = [16, 16] 


pysqldf = lambda q: sqldf(q, globals())


In [3]:
def replace_space_in_col_names(df, c = " "):
    new_cols = [x.replace(c,"_") for x in df.columns]
    df.columns = new_cols

In [4]:
#outpatients stats
outpatients_df = pd.read_csv("../web_data/hc_july2017/Outpatient Procedures - Volume.csv")
outpatients_df = outpatients_df[["Provider_ID","Hospital_Name","Cardiovascular","Respiratory"]]
#91.30530973451327

#hospital returns
hospital_returns_df = pd.read_csv("../web_data/hc_july2017/Hospital Returns - Hospital.csv")
replace_space_in_col_names(hospital_returns_df)
hospital_returns_df = hospital_returns_df[["Provider_ID", "Hospital_Name","Measure_Name","Compared_to_National"]]
hospital_returns_df = hospital_returns_df[hospital_returns_df.Measure_Name.isin(['Hospital return days for heart attack patients','Hospital return days for heart failure patients','Rate of readmission after discharge from hospital (hospital-wide)'])]
hospital_returns_df_pivot = hospital_returns_df.pivot_table(index=["Provider_ID","Hospital_Name"],columns="Measure_Name",values="Compared_to_National",aggfunc=lambda x : x).reset_index()
replace_space_in_col_names(hospital_returns_df_pivot)
hospital_returns_df_pivot.rename(columns={"Rate_of_readmission_after_discharge_from_hospital_(hospital-wide)":"Rate_readmission_after_discharge"},inplace=True)
#100

#safety and cost reduction scores
'''
The HVBP program is part of CMS’ long-standing effort to link Medicare’s payment system to quality. 
The program implements value-based purchasing to the payment system that accounts for the largest share of 
Medicare spending, affecting payment for inpatient stays in over 3,500 hospitals across the country. 
Hospitals are paid for inpatient acute care services based on the quality of care, not just quantity of the services they provide. The Fiscal Year 2016 HVBP program adjusts hospitals’ payments based on their performance on five domains that reflect hospital quality. The domains consiste of measures for Safety, Patient Experience of Care, Clinical Care Outcomes, Perinatal Outcomes, Immunizations, and Efficiency. The Total Performance Score (TPS) is comprised of the scores from the following domains: Clinical Care -Process domain score (weighted as 5% of the TPS), Clinical Care – Outcomes (weighted as 25% of the TPS), the Patient- and Caregive-Centered Experience of Care/Care Coordination domain score (weighted as 25 percent of the TPS), the Safety domain score (weighted as 20 percent of the TPS), 
and the Efficiency and Cost Reduction domain score (weighted as 25 percent of the TPS).
'''
safety_cost_df = pd.read_csv("../web_data/hc_july2017/hvbp_tps_11_10_2016.csv")
replace_space_in_col_names(safety_cost_df)
safety_cost_df.columns
safety_cost_df = safety_cost_df[["Provider_Number","Hospital_Name","Weighted_Safety_Domain_Score","Weighted_Efficiency_and_Cost_Reduction_Domain_Score","Total_Performance_Score"]]
safety_cost_df["Weighted_Safety_Domain_Score"] = safety_cost_df["Weighted_Safety_Domain_Score"].apply(lambda x : float('nan') if x == "Not Available" else float(x))
safety_cost_df["Total_Performance_Score"] = safety_cost_df["Total_Performance_Score"].apply(lambda x : float('nan') if x == "Not Available" else float(x))
safety_cost_df["Weighted_Efficiency_and_Cost_Reduction_Domain_Score"] = safety_cost_df["Weighted_Efficiency_and_Cost_Reduction_Domain_Score"].apply(lambda x : float('nan') if x == "Not Available" else float(x))
safety_cost_df["Weighted_Safety_Domain_Score"].fillna(safety_cost_df["Weighted_Safety_Domain_Score"].median())
safety_cost_df["Total_Performance_Score"].fillna(safety_cost_df["Total_Performance_Score"].median())
safety_cost_df["Weighted_Efficiency_and_Cost_Reduction_Domain_Score"].fillna(safety_cost_df["Weighted_Efficiency_and_Cost_Reduction_Domain_Score"].median())

safety_cost_df.rename(columns={"Provider_Number":"Provider_ID"},inplace=True)
#65.37610619469027


#Payment and Value Care
payment_value_care_df = pd.read_csv("../web_data/hc_july2017/Payment and Value of Care - Hospital.csv")
replace_space_in_col_names(payment_value_care_df)
payment_value_care_df.columns
payment_value_care_df = payment_value_care_df.rename(columns={'Hospital_name': 'Hospital_Name'})
payment_value_care_df = payment_value_care_df[payment_value_care_df.Payment_measure_name.isin(['Payment for heart attack patients','Payment for heart failure patients'])]
payment_value_care_df = payment_value_care_df[["Provider_ID","Hospital_Name","Payment_measure_name","Payment","Value_of_care_category"]]
payment_value_care_df = payment_value_care_df[payment_value_care_df.Payment != 'Not Available']
payment_value_care_df["Payment"] = payment_value_care_df.Payment.apply(lambda x : float(x.split("$")[1].replace(',','')))
payment_value_care_df
#pd.pivot_table( payment_value_care_df, index=["Provider_ID","Hospital_Name"],columns=["Payment_measure_name","Value_of_care_category"]).reset_index().fillna(0).columns
payment_value_care_df_pivot = pd.pivot_table(payment_value_care_df,index=["Provider_ID","Hospital_Name"],columns=["Payment_measure_name"],values=["Value_of_care_category","Payment"],aggfunc=[lambda x : ''.join(x), np.sum]).reset_index()
payment_value_care_df_pivot.columns = [''.join(col).strip() for col in payment_value_care_df_pivot.columns.values]
payment_value_care_df_pivot.rename(columns={'<lambda>Value_of_care_categoryPayment for heart attack patients':'value_of_care_heart_attack_patients',
                                           '<lambda>Value_of_care_categoryPayment for heart failure patients':'value_of_care_heart_failure_patients',
                                           'sumPaymentPayment for heart attack patients':'payment_for_heart_attack_patients',
                                           'sumPaymentPayment for heart failure patients':'payment_for_heart_failure_patients',},inplace=True)
payment_value_care_df_pivot

#100

#Survey Data - Communication about the medicines 
comm_survey_df = pd.read_csv("../web_data/hc_july2017/hvbp_hcahps_11_10_2016.csv")
replace_space_in_col_names(comm_survey_df)
comm_survey_df.rename(columns={"Provider_Number":"Provider_ID"},inplace=True)

comm_survey_df["Communication_about_Medicines_Performance_Rate"] = comm_survey_df["Communication_about_Medicines_Performance_Rate"].apply(lambda x : float('nan') if x == 'Not Available' else float(x))
comm_survey_df.fillna(comm_survey_df["Communication_about_Medicines_Performance_Rate"] .median())

comm_survey_df["Overall_Rating_of_Hospital_Performance_Rate"] = comm_survey_df["Overall_Rating_of_Hospital_Performance_Rate"].apply(lambda x : float('nan') if x == 'Not Available' else float(x))
comm_survey_df.fillna(comm_survey_df["Overall_Rating_of_Hospital_Performance_Rate"] .median())
comm_survey_df.rename(columns= {"Overall_Rating_of_Hospital_Performance_Rate" : "Overall_Communication_Rate"},inplace=True)

comm_survey_df = comm_survey_df[["Provider_ID","Hospital_Name","Communication_about_Medicines_Performance_Rate","Overall_Communication_Rate"]]
#65.37610619469027

#General Information
hospital_gi_df = pd.read_csv("../web_data/hc_july2017/Hospital General Information_with_lat_long.csv",encoding = "ISO-8859-1")
replace_space_in_col_names(hospital_gi_df,".")
hospital_gi_df["ZIP_Code"] = hospital_gi_df["ZIP_Code"].astype(str)
hospital_gi_df["Hospital_overall_rating"] = hospital_gi_df["Hospital_overall_rating"].apply(lambda x : 2.5 if x == "Not Available" else float(x) )
hospital_gi_df = hospital_gi_df[["Provider_ID","Hospital_Name","ZIP_Code","Hospital_Ownership","Emergency_Services","Meets_criteria_for_meaningful_use_of_EHRs","Hospital_overall_rating","Mortality_national_comparison","Safety_of_care_national_comparison","Readmission_national_comparison","Patient_experience_national_comparison","Effectiveness_of_care_national_comparison","Latitude","Longitude"]]


#Census data
zip_df = pd.read_csv("../data/FromNCU/data/superzip.csv")
zip_df["zipcode"] = zip_df["zipcode"].astype(str)
zip_df["college"] = zip_df["college"].apply(lambda x : x.split("%")[0])
zip_df["college"]
zip_df = zip_df[["zipcode","city","state","adultpop","households","college","income"]]
zip_df.columns
hospital_gi_df_with_census = pd.merge(hospital_gi_df,zip_df[["zipcode","adultpop","households","college","income"]],left_on="ZIP_Code",right_on="zipcode")
len(hospital_gi_df_with_census)



4520

In [5]:
len(safety_cost_df)/len(hospital_gi_df_with_census) * 100

65.37610619469027

In [7]:
#from functools import reduce
dfs = [hospital_gi_df_with_census,outpatients_df,safety_cost_df,payment_value_care_df,comm_survey_df]
#third_party_final_df = reduce(lambda left,right: pd.merge(left,right,on='Provider_ID'), dfs)

third_party_final_df = pd.merge(hospital_gi_df_with_census,outpatients_df,on='Provider_ID',how='left')
third_party_final_df = pd.merge(third_party_final_df,safety_cost_df,on='Provider_ID',how='left')
third_party_final_df = pd.merge(third_party_final_df,payment_value_care_df_pivot,on='Provider_ID',how='left')
third_party_final_df = pd.merge(third_party_final_df,comm_survey_df,on='Provider_ID',how='left')

len(third_party_final_df)
third_party_final_df = third_party_final_df.fillna(-1)
third_party_final_df = third_party_final_df.drop(["Hospital_Name_x","Hospital_Name_y"],axis=1)

coverage_df = pd.DataFrame(columns=["col_name","coverage_percentage"])
for c in list(third_party_final_df.columns):
    print(c)
    na_values = len(third_party_final_df[third_party_final_df[c] == -1])
    covered_values_perc = ((len(third_party_final_df) - na_values)/len(third_party_final_df)) * 100
    coverage_df = coverage_df.append({'col_name':c,'coverage_percentage':covered_values_perc},ignore_index=True)

#third_party_final_df[third_party_final_df["Meets_criteria_for_meaningful_use_of_EHRs"] == -1]
coverage_df.to_clipboard(index=False)
#third_party_final_df.to_csv("Hospitals_Features/non_ahd_data.csv",index=False)

Provider_ID
ZIP_Code
Hospital_Ownership
Emergency_Services
Meets_criteria_for_meaningful_use_of_EHRs
Hospital_overall_rating
Mortality_national_comparison
Safety_of_care_national_comparison
Readmission_national_comparison
Patient_experience_national_comparison
Effectiveness_of_care_national_comparison
Latitude
Longitude
zipcode
adultpop
households
college
income
Cardiovascular
Respiratory
Weighted_Safety_Domain_Score
Weighted_Efficiency_and_Cost_Reduction_Domain_Score
Total_Performance_Score
value_of_care_heart_attack_patients
value_of_care_heart_failure_patients
payment_for_heart_attack_patients
payment_for_heart_failure_patients
Hospital_Name
Communication_about_Medicines_Performance_Rate
Overall_Communication_Rate


In [8]:
#third_party_final_df[third_party_final_df.Provider_ID.isin(safety_cost_df.Provider_ID)]
pd.merge(hospital_gi_df_with_census,safety_cost_df,on='Provider_ID')

,Provider_ID,Hospital_Name_x,ZIP_Code,Hospital_Ownership,Emergency_Services,Meets_criteria_for_meaningful_use_of_EHRs,Hospital_overall_rating,Mortality_national_comparison,Safety_of_care_national_comparison,Readmission_national_comparison,...,Longitude,zipcode,adultpop,households,college,income,Hospital_Name_y,Weighted_Safety_Domain_Score,Weighted_Efficiency_and_Cost_Reduction_Domain_Score,Total_Performance_Score
0,220024,HOLYOKE MEDICAL CENTER,1040,Voluntary non-profit - Private,Yes,Y,2.0,Same as the national average,Below the national average,Below the national average,...,-72.6262,1040,24509,9530,16.93256,47.24359,HOLYOKE MEDICAL CENTER,14.000000,0.000000,35.583333
1,220015,"COOLEY DICKINSON HOSPITAL INC,THE",1060,Voluntary non-profit - Private,Yes,Y,3.0,Same as the national average,Below the national average,Same as the national average,...,-72.6313,1060,10508,2740,52.02703,72.37566,"COOLEY DICKINSON HOSPITAL INC,THE",10.800000,2.500000,30.550000
2,220030,BAYSTATE WING HOSPITAL AND MEDICAL CENTERS,1069,Voluntary non-profit - Other,Yes,Y,4.0,Above the national average,NaN,Same as the national average,...,-72.3288,1069,6746,2630,14.20101,67.69183,BAYSTATE WING HOSPITAL AND MEDICAL CENTERS,NaN,3.125000,25.520833
3,220050,BAYSTATE MARY LANE HOSPITAL,1082,Voluntary non-profit - Private,Yes,Y,4.0,Same as the national average,Same as the national average,Same as the national average,...,-72.2583,1082,6688,2664,13.83074,59.91946,BAYSTATE MARY LANE HOSPITAL,NaN,0.000000,33.750000
4,220065,NOBLE HOSPITAL,1085,Voluntary non-profit - Private,Yes,Y,4.0,Same as the national average,Same as the national average,Same as the national average,...,-72.7495,1085,25900,10230,24.37452,72.51949,NOBLE HOSPITAL,NaN,3.125000,35.000000
5,220066,MERCY MEDICAL CTR,1104,Voluntary non-profit - Private,Yes,Y,3.0,Same as the national average,Above the national average,Below the national average,...,-72.5778,1104,14338,5555,12.10769,49.85225,MERCY MEDICAL CTR,5.200000,0.000000,26.700000
6,220046,BERKSHIRE MEDICAL CENTER INC - 1,1201,Voluntary non-profit - Other,Yes,Y,4.0,Same as the national average,Above the national average,Below the national average,...,-73.2471,1201,33416,12341,21.02586,60.92893,BERKSHIRE MEDICAL CENTER INC,12.333333,0.000000,33.583333
7,220016,BAYSTATE FRANKLIN MEDICAL CENTER,1301,Voluntary non-profit - Private,Yes,Y,3.0,Same as the national average,Same as the national average,Same as the national average,...,-72.6236,1301,12897,4496,24.2537,60.94854,BAYSTATE FRANKLIN MEDICAL CENTER,NaN,18.750000,41.875000
8,220098,NASHOBA VALLEY MEDICAL CENTER,1432,Proprietary,Yes,Y,3.0,Same as the national average,Same as the national average,Same as the national average,...,-71.5788,1432,5671,1797,24.14036,81.02936,NASHOBA VALLEY MEDICAL CENTER,NaN,0.000000,30.000000
9,220095,HEYWOOD HOSPITAL,1440,Voluntary non-profit - Private,Yes,Y,3.0,Same as the national average,Same as the national average,Same as the national average,...,-71.9898,1440,14559,5209,15.07659,61.47158,HEYWOOD HOSPITAL,10.400000,0.000000,27.150000


In [9]:
pd.DataFrame(third_party_final_df.columns).to_clipboard(index=False)
#' , '.join(hospital_gi_df_with_census.Hospital_Ownership.unique())


In [10]:
list(third_party_final_df.Communication_about_Medicines_Performance_Rate.unique())
third_party_final_df.Communication_about_Medicines_Performance_Rate.describe()


count    4520.000000
mean       37.962263
std        31.793713
min        -1.000000
25%        -1.000000
50%        58.870000
75%        64.410000
max        90.560000
Name: Communication_about_Medicines_Performance_Rate, dtype: float64

In [11]:
payment_value_care_df.head(4)
#payment_value_care_df.pivot_table(index=["Provider_ID","Hospital_Name"],columns=["Payment_measure_name"],values=["Payment","Value_of_care_category"],aggfunc=lambda x : ''.join(x)).reset_index()

,Provider_ID,Hospital_Name,Payment_measure_name,Payment,Value_of_care_category
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,Payment for heart attack patients,22178.0,Average mortality and average payment
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,Payment for heart failure patients,16525.0,Average mortality and average payment
4,10005,MARSHALL MEDICAL CENTER SOUTH,Payment for heart attack patients,22133.0,Average mortality and average payment
5,10005,MARSHALL MEDICAL CENTER SOUTH,Payment for heart failure patients,16321.0,Worse mortality and average payment


In [ ]:
len(third_party_final_df)

In [12]:
#check the difference between Defenitive Health Care and AHD
def_df = pd.read_excel("../data/DefinitiveHealthData.xlsx",0)
replace_space_in_col_names(def_df)
def_df["Provider_Number"] = def_df["Provider_Number"].astype(str)
def_df["Provider_Number"] = def_df["Provider_Number"].apply(lambda x : x.zfill(6))
def_df= def_df[~def_df["Provider_Number"].str.contains('nan')]
def_df= def_df[~def_df["Provider_Number"].str.contains("\\*")]
def_df= def_df[~def_df["Provider_Number"].str.contains("Closed")]

def_df_difference = def_df[~def_df.Provider_Number.isin(third_party_final_df.Provider_ID.astype(str).unique())]
def_df_difference.to_csv("DefinitiveAhdDifference.csv",index=False)
#third_party_final_df.Provider_ID

In [13]:
print(len(def_df))
print(len(def_df_difference))
print(len(third_party_final_df))
print(len(def_df[def_df.Provider_Number.isin(third_party_final_df.Provider_ID.astype(str).unique())]))

6357
2564
4520
3793


In [ ]:
def_df_difference["Provider_Number"]